# Retrieving additional census data for AGI per Washington zip code

This data was obtained from the data.census.gov website.  It was downloaded in Excel format and some data cleaning, specifically removing of unneeded columns and footnotes, was performed in Excel.  Here we will continue scrubbing the data to get to a list of zip codes in Washington with their corresponding value for Adjusted gross income per tax return filed.


In [1]:
!pip install xlrd
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('data/agi_zip_code.xlsx',skiprows=3)
df.head(20)

,ZIP\ncode [1],Size of adjusted gross income,Number of returns,Adjusted gross income (AGI) [6]
0,0,Total,3579250.0,326628023.0
1,0,"$1 under $25,000",917680.0,11770347.0
2,0,"$25,000 under $50,000",841340.0,30781842.0
3,0,"$50,000 under $75,000",542860.0,33473947.0
4,0,"$75,000 under $100,000",375730.0,32614295.0
5,0,"$100,000 under $200,000",627660.0,86306580.0
6,0,"$200,000 or more",273980.0,131681012.0
7,NaN,NaN,NaN,NaN
8,98001,NaN,16940.0,1302277.0
9,98001,"$1 under $25,000",3990.0,50371.0


In [3]:
df.columns

Index(['ZIP\ncode [1]', 'Size of adjusted gross income', 'Number of returns',
       'Adjusted gross income (AGI) [6]'],
      dtype='object')

## Data Cleaning

* The data includes multiple rows for each zip code, but the first row for each zip code is the total  So we will remove all duplicates except the first instance.
* Remove the rows with zip code of 0, which represent grand totals for the state.
* Remove the "Size of adjusted gross income" column
* Set the index to the Zip Code
* Rename the columns

In [4]:
df.drop_duplicates(subset='ZIP\ncode [1]',keep='first', inplace=True)

In [5]:
df=df[df['ZIP\ncode [1]'] != 0]

In [6]:
df.drop(labels='Size of adjusted gross income', axis=1, inplace=True)

In [7]:
df=df.dropna()

In [8]:
df.columns=['ZIP','returns','AGI']

In [9]:
df=df.set_index('ZIP')

In [10]:
df.sort_values('AGI', ascending=False)

,returns,AGI
ZIP,,
98004,18840.0,6309943.0
98052,33650.0,5647579.0
99999,60440.0,5643123.0
98033,19980.0,4917685.0
98040,12650.0,4405904.0
...,...,...
98834,90.0,5196.0
98552,140.0,4777.0
99356,100.0,4746.0


## Feature Engineering

We currently have the total Adjusted Gross Income for each zip code.  However this does not take into consideration the population or the number of return filed in each zip code so that data will be skewed to more dense areas.  We will create a column for AGI per Return to give us a better estimator of which zip codes are lived in by the wealthiest people.

In [11]:
df['AGIperReturn']=df['AGI']/df['returns']

In [12]:
df.sort_values('AGIperReturn', ascending=False)

,returns,AGI,AGIperReturn
ZIP,,,
98039,1540.0,1808237.0,1174.179870
98040,12650.0,4405904.0,348.292806
98004,18840.0,6309943.0,334.922665
98112,12660.0,4202173.0,331.925197
98164,110.0,33940.0,308.545455
...,...,...,...
99140,130.0,4620.0,35.538462
99321,400.0,13767.0,34.417500
98552,140.0,4777.0,34.121429


In [13]:
top_30_zips = df.sort_values('AGIperReturn', ascending=False)[:30]

In [14]:
top_30_zips.head()

,returns,AGI,AGIperReturn
ZIP,,,
98039,1540.0,1808237.0,1174.179870
98040,12650.0,4405904.0,348.292806
98004,18840.0,6309943.0,334.922665
98112,12660.0,4202173.0,331.925197
98164,110.0,33940.0,308.545455


## Export file

We now have a listing of the top 30 zip codes in the state of Washington as calculated by the Adjusted Gross Income per tax return for each zip code.

In [15]:
top_30_zips.to_csv('data/top_30_zips.csv')